# AoC 2024 - Day 9

<https://adventofcode.com/2024/day/9>

In [1]:
from icecream import ic
import time

In [2]:
use_test = True  # Comment out for using actual puzzle input

## Part 1

In [3]:
test_data1 = """12345
"""

test_data2 = """2333133121414131402
"""

try:
    use_test
except NameError:
    use_test = False
    ic("use_test was undefined - forcing to", use_test)
    
if use_test:
    assert test_data1 != None
    assert test_data2 != None

In [4]:
# Read the puzzle input into a list of strings, one per line
#
if use_test:
    input_lines = test_data2.splitlines()
else:
    with open("input_day09_gmacario.txt", 'r') as file:
        input_lines = [line.rstrip() for line in file]

ic(time.ctime())
if use_test:
    ic(input_lines)

ic| time.ctime(): 'Mon Dec  9 21:01:52 2024'
ic| input_lines: ['2333133121414131402']


In [5]:
def convert_input_to_string(input_str:str, verbose=False) -> str:
    # ic(input_str)
    result = ""
    is_free_space = False
    file_id = 0
    for ch in input_str.rstrip():
        sz = int(ch)
        if is_free_space:
            if verbose:
                print(f"DEBUG: {sz} block(s) of free space")
            result += "." * sz
        else:
            if verbose:
                print(f"DEBUG: {sz} block(s) file (file_id {file_id})")
            # result += f"({str(file_id)})" * sz
            result += str(file_id) * sz
            file_id += 1
        is_free_space = not is_free_space
    return result[:]

In [6]:
def convert_input_to_map(input_str:str, verbose=False) -> list:
    # ic(input_str)
    result = list()
    is_free_space = False
    file_id = 0
    for ch in input_str.rstrip():
        sz = int(ch)
        if is_free_space:
            if verbose:
                print(f"DEBUG: {sz} block(s) of free space")
            result.extend(["."] * sz)
        else:
            if verbose:
                print(f"DEBUG: {sz} block(s) file (file_id {file_id})")
            result.extend([file_id] * sz)
            file_id += 1
        is_free_space = not is_free_space
    return result[:]

In [7]:
convert_input_to_string(test_data1, verbose=True)

DEBUG: 1 block(s) file (file_id 0)
DEBUG: 2 block(s) of free space
DEBUG: 3 block(s) file (file_id 1)
DEBUG: 4 block(s) of free space
DEBUG: 5 block(s) file (file_id 2)


'0..111....22222'

In [8]:
x = convert_input_to_map(test_data2, verbose=True)
print(x)

DEBUG: 2 block(s) file (file_id 0)
DEBUG: 3 block(s) of free space
DEBUG: 3 block(s) file (file_id 1)
DEBUG: 3 block(s) of free space
DEBUG: 1 block(s) file (file_id 2)
DEBUG: 3 block(s) of free space
DEBUG: 3 block(s) file (file_id 3)
DEBUG: 1 block(s) of free space
DEBUG: 2 block(s) file (file_id 4)
DEBUG: 1 block(s) of free space
DEBUG: 4 block(s) file (file_id 5)
DEBUG: 1 block(s) of free space
DEBUG: 4 block(s) file (file_id 6)
DEBUG: 1 block(s) of free space
DEBUG: 3 block(s) file (file_id 7)
DEBUG: 1 block(s) of free space
DEBUG: 4 block(s) file (file_id 8)
DEBUG: 0 block(s) of free space
DEBUG: 2 block(s) file (file_id 9)
[0, 0, '.', '.', '.', 1, 1, 1, '.', '.', '.', 2, '.', '.', '.', 3, 3, 3, '.', 4, 4, '.', 5, 5, 5, 5, '.', 6, 6, 6, 6, '.', 7, 7, 7, '.', 8, 8, 8, 8, 9, 9]


In [9]:
# convert_input_to_map(test_data2, verbose=False)

In [10]:
disk_map = convert_input_to_map(test_data2)
print(disk_map)

[0, 0, '.', '.', '.', 1, 1, 1, '.', '.', '.', 2, '.', '.', '.', 3, 3, 3, '.', 4, 4, '.', 5, 5, 5, 5, '.', 6, 6, 6, 6, '.', 7, 7, 7, '.', 8, 8, 8, 8, 9, 9]


In [11]:
def compute_checksum(disk_map: list) -> int:
    result = 0
    position = 0
    for block in disk_map:
        if block != ".":
            result += block * position
        position += 1
    return result

In [12]:
def sort_disk_map(disk_map: list) -> list:
    # ic(len(disk_map))
    for pos in range(len(disk_map)-1,-1,-1):
        if disk_map[pos] != ".":
            # ic(pos)
            for pos2 in range(pos):
                if disk_map[pos2] == ".":
                    disk_map[pos], disk_map[pos2] = disk_map[pos2], disk_map[pos]
                else:
                    pass
        else:
            pass
    
    # print(disk_map)
    return disk_map

In [13]:
# input_lines

In [14]:
disk_map = convert_input_to_map(input_lines[0],verbose=False)
# print(disk_map)

In [15]:
compute_checksum(disk_map)

4116

In [16]:
def solve_part1(disk_map):
    sort_disk_map(disk_map)
    result = compute_checksum(disk_map)
    return result

tm_begin = time.time()
part1_result = solve_part1(disk_map)
tm_end = time.time()
ic("elapsed=", tm_end - tm_begin, "result=", part1_result)
print("Day 09 Part 1 RESULT:")
print(part1_result)

ic| "elapsed=": 'elapsed='
    tm_end - tm_begin: 0.0002110004425048828
    "result=": 'result='
    part1_result: 1928


Day 09 Part 1 RESULT:
1928


## Part 2

In [17]:
disk_map = convert_input_to_map(input_lines[0],verbose=False)
print(disk_map)

[0, 0, '.', '.', '.', 1, 1, 1, '.', '.', '.', 2, '.', '.', '.', 3, 3, 3, '.', 4, 4, '.', 5, 5, 5, 5, '.', 6, 6, 6, 6, '.', 7, 7, 7, '.', 8, 8, 8, 8, 9, 9]


In [18]:
# Find the largest file_id number
highest_file_id = max(((id if id != "." else -1) for id in disk_map))
ic(highest_file_id)
# num_items_with_file_id = sum((id == highest_file_id for id in disk_map))
num_items_with_file_id = disk_map.count(highest_file_id)
ic(num_items_with_file_id)

first_empty_pos = disk_map.index(".")
ic(first_empty_pos)

# disk_map = disk_map[0:first_empty_pos].extend([highest_file_id] * num_items_with_file_id)
# print(disk_map)

# next_full_pos = disk_map.search()

ic| highest_file_id: 9
ic| num_items_with_file_id: 2
ic| first_empty_pos: 2


2

In [19]:
assert False

AssertionError: 

In [ ]:
# ic(input_map)
# ic(get_antenna_frequencies(input_map))

In [ ]:
# set_0 = find_antinodes(input_map, "0", multiple=True)
# _ = ic(len(set_0), set_0)

In [ ]:
# set_A = find_antinodes(input_map, "A", multiple=True)
# _ = ic(len(set_A), set_A)

In [ ]:
# set_total = set_0.union(set_A)
# _ = ic(len(set_total), set_total)

In [ ]:
def solve_part2(input_map):
    result = set()
    for antenna_freq in get_antenna_frequencies(input_map):
        x = find_antinodes(input_map, antenna_freq, multiple=True)
        # ic(x)
        result = result.union(x)
        # ic(x, result)
    return len(result)

tm_begin = time.time()
part2_result = solve_part2(input_map)
tm_end = time.time()
ic("elapsed=", tm_end - tm_begin, "result=", part2_result)
print("Day 09 Part 2 RESULT:")
print(part2_result)